In [1]:
config = {
    "pretrained_model": "bert-base-uncased",
    "tokenizer": "bert-base-uncased",
    "max_seq_length": 512,
    "batch_size": 16,
    "lr": 2e-5,
    "epochs": 10,
    "device": "cpu",
    "gpu_ids": "3",
    "seed": 2020,
    "fp16": False,
    "loss_scale": 0,
    "gradient_accumulation_steps":1,
    "warmup_proportion": 0.1,
    "gradient_accumulation_steps": 1,
    "num_labels": 2,
    "is_multilabel": False,
    "valid_metric": "f1",
    "model_save_dir": "../checkpoints/bert_uncased_512_biocaster_binary_20200903/",
    "patience": 4,
}

In [2]:
import sys
sys.path.append("../")
from beta_nlp.utils.data_util import biocaser2text
data_file = "/home/zm324/workspace/doc_cls/datasets/biocaster/BioCaster.3.xml"
data_df = biocaser2text(data_file)
data_df["source"] = "Biocaster"
data_df.head(3)

parse biocaser data from /home/zm324/workspace/doc_cls/datasets/biocaster/BioCaster.3.xml, docs number:1003, lablels number:1003


,docs,labels,source
0,\nBird Flu Outbreak Drill Spooks Manitoba Town...,0,Biocaster
1,\nTyphoid outbreak in Agusan del Sur town unde...,1,Biocaster
2,\n Typhoid Outbreak In Central Nepal November...,1,Biocaster


In [3]:
from sklearn.utils import shuffle
data_df = shuffle(data_df).reset_index()
data_df["flag"]=None
train_index = int(len(data_df.index)*0.8)
valid_index = train_index + round(len(data_df.index)*0.1)
data_df.iloc[:train_index]["flag"]="train"
data_df.iloc[train_index:valid_index]["flag"]="valid"
data_df.iloc[valid_index:]["flag"]="test"

<ipython-input-3-79406d09754d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df.iloc[:train_index]["flag"]="train"
<ipython-input-3-79406d09754d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df.iloc[train_index:valid_index]["flag"]="valid"
<ipython-input-3-79406d09754d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [4]:
train_set = data_df[data_df["flag"]=="train"]
dev_set = data_df[data_df["flag"]=="valid"]
test_set = data_df[data_df["flag"]=="test"]

In [5]:
len(train_set.index),len(dev_set.index),len(test_set.index)

(802, 100, 101)

In [6]:
from beta_nlp.models.bert_cls import BertModel
cls = BertModel(config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [7]:
len(train_set.index),len(dev_set.index),len(test_set.index)

(802, 100, 101)

In [ ]:
cls.train(train_set,dev_set)

Evaluating:   0%|          | 0/7 [00:00<?, ?it/s]

Execute [train_an_epoch] method costing 534412.57 ms
[Epoch 0] loss: 33.067089676856995



Evaluating: 100%|██████████| 7/7 [00:17<00:00,  2.56s/it]


Execute [eval] method costing 17913.81 ms
------------------------------------------------------------
[Epoch 0]performance on validation set
+----+-----------+----------+
|    | metrics   |   values |
|----+-----------+----------|
|  0 | accuracy  | 0.86     |
|  1 | precision | 0.692308 |
|  2 | recall    | 0.931034 |
|  3 | f1        | 0.794118 |
+----+-----------+----------+
------------------------------------------------------------


Evaluating:   0%|          | 0/7 [00:00<?, ?it/s]

...


Evaluating:   0%|          | 0/7 [00:00<?, ?it/s]

Execute [train_an_epoch] method costing 544577.49 ms
[Epoch 6] loss: 1.2391234126407653



Training:   0%|          | 0/51 [00:00<?, ?it/s]

Execute [eval] method costing 15016.49 ms



Training:  12%|█▏        | 6/51 [01:01<07:38, 10.18s/it]

In [ ]:
cls.test(test_set)

In [ ]:
test_set.iloc[1:10]

In [ ]:
cls.predict(test_set.iloc[1:10])